## 톤업, 컬러 제품 포함하여 구매한 고객의 구매 패턴

- 가설:  톤업 또는 컬러 제품을 포함하여 구매하는 고객들은 아닌 고객보다 주문 별 구매 수량이 많을 것이다. 왜냐하면 풀컬러 제품은 다른 제품과 믹스매치해서 사용하기 때문에

In [44]:
import pandas as pd

In [45]:
# 분석하고 싶은 제품 모델명 리스트
toneup=pd.read_csv("../../data/product/toneup.csv",encoding='ms949')

In [46]:
# 2020년 주문내역
s20=pd.read_csv("../../data/sales/2020.csv",encoding='ms949') # 2020 01 01 ~ 2020 60 30
s2007=pd.read_csv("../../data/sales/200709.csv",encoding='ms949') # 2020 07 01 ~ 2020 07 09

In [47]:
# 주문서 합치기
s=pd.concat([s20,s2007])

### 주문서 테이블 구조

- row는 상품 1개의 정보로 되어 있음 
- 주문번호 컬럼과 모델명을 합쳐서 unique로 사용할 수 있음 (주문 하나에 여러상품이 포함될 경우 각 상품 row는 같은 주문번호를 가지기 때문에 주문번호 단독으로 unique로 쓸수 없음)

In [144]:
#샘플
data = {
    '주문번호' : [1,1,1,2,3],
    '고객명' : ['홍길동','홍길동','홍길동','김나나','이도도'],
    '모델명' :['바나나','사과','포도','바나나','바나나'],
    '수량' : [1,3,2,1,3]
}
df = pd.DataFrame(data)
df

,주문번호,고객명,모델명,수량
0,1,홍길동,바나나,1
1,1,홍길동,사과,3
2,1,홍길동,포도,2
3,2,김나나,바나나,1
4,3,이도도,바나나,3


위 샘플 테이블 구조로 되어 있는데, 위 테이블을 해석하면 주문 고객은 홍길동, 김나나, 이도도로 세명이고
홍길동이 주문한 주문번호 1번에는 바나나1개, 사과3개, 포도 2개, 김나나가 주문한 주문번호 2번에는 바나나 1개, 이도도가 주문한 주문번호 3번에는 바나나3개 있음

## preprocessing

In [48]:
s=s[s['결제상태'].str.contains('입금완료')]

In [49]:
s['입금확인일(결제일)']=pd.to_datetime(s['입금확인일(결제일)'])
s['결제월']=s['입금확인일(결제일)'].dt.month

In [50]:
s.loc[s['모델명'].isnull(),'모델명'] = s.loc[s['모델명'].isnull(),'옵션 관리코드']

In [51]:
toneup_model=list(toneup['모델명'])
print("톤업,컬러 제품 squ:",len(toneup_model))

톤업,컬러 제품 squ: 125


## aggfunc

- 주문내역에서 판매 된 제품이 toneup_model 리스트에 포함된 제품인지 확인하는 컬럼 추가하여 톤업 포함 주문과 아닌 주문 구별

In [54]:
s['톤업여부']=s['모델명'].isin(toneup_model)

In [67]:
s1=s[['주문번호','모델명','주문품목 수량','주문품목 결제금액','결제월','톤업여부']]
s2=s[['주문번호','결제금액(주문전체)','결제월','톤업여부']]

In [83]:
# 톤업제품 포함하여 주문한 주문번호 리스트
toneup_in_ordernum=list(s1.loc[s1['톤업여부'] == True,'주문번호'])
t_in=s1[s1['주문번호'].isin(toneup_in_ordernum)] # 톤업 주문 포함한 주문테이블
t_not=s1[~s1['주문번호'].isin(toneup_in_ordernum)] # 톤업 주문 포함하지 않은 주문테이블

In [117]:
t_in2=s2[s2['주문번호'].isin(toneup_in_ordernum)] # 톤업 주문 포함한 주문테이블
t_not2=s2[~s2['주문번호'].isin(toneup_in_ordernum)] # 톤업 주문 포함하지 않은 주문테이블

### 1) 구매건당 주문 수량 비교

In [110]:
t1=t_in.groupby('결제월')['주문품목 수량'].sum()
t2=t_in.groupby('결제월')['주문번호'].nunique()
t3=round(t1/t2,2)
t_in_num=pd.concat([t1,t2,t3],axis=1) # 월별 주문당 구매제품 수량

In [111]:
t1=t_not.groupby('결제월')['주문품목 수량'].sum()
t2=t_not.groupby('결제월')['주문번호'].nunique()
t3=round(t1/t2,2)
t_not_num=pd.concat([t1,t2,t3],axis=1) # 월별 주문당 구매제품 수량

### 2) 구매건당 객단가 비교

In [121]:
# 주문당 전체 금액 확인하기 위한 테이블 
t_in2=t_in2.drop_duplicates('주문번호').reset_index().drop(['index'],axis=1)
t_not2=t_not2.drop_duplicates('주문번호').reset_index().drop(['index'],axis=1)

In [130]:
t1=t_in2.groupby('결제월')['결제금액(주문전체)'].sum()
t2=t_in2.groupby('결제월')['주문번호'].nunique()
t3=round(t1/t2,2)
t_in_order=pd.concat([t1,t2,t3],axis=1) # 월별 주문당 객단가 

In [131]:
t1=t_not2.groupby('결제월')['결제금액(주문전체)'].sum()
t2=t_not2.groupby('결제월')['주문번호'].nunique()
t3=round(t1/t2,2)
t_not_order=pd.concat([t1,t2,t3],axis=1) # 월별 주문당 객단가 

### 3) 구매건당 톤업 / 컬러 제품 포함 갯수

- 주문번호에서 톤업여부 True 인 상품 컬럼만 추출

In [150]:
t1=t_in[t_in['톤업여부']].groupby('결제월')['주문품목 수량'].sum()
t2=t_in[t_in['톤업여부']].groupby('결제월')['주문번호'].nunique()
t3=round(t1/t2,2)
t_in_order2=pd.concat([t1,t2,t3],axis=1) # 월별 주문당 구매제품 수량